In [3]:
# %load F:/IIT/1st Sem/Hardware sec of IOT/Assignment 3/assignment3/assignment3/CPA/exp/CPA_for__first_byte.py
import pandas as pd
import numpy as np
df = pd.read_csv (r'AES_Power_Trace.csv')
df1=df.to_numpy()
print (df1)
from scipy.stats import pearsonr
from numpy import unravel_index

traces=df1[0:, 2:2000]
#Getting the plaintext part
plaintexts=df1[:,0]
#Getting Ciphertext part
ciphertexts=df1[:,1]
InvSbox = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)
cipher9=np.empty([8939,256])
#For Hamming weight
def number_of_ones(n):
    c = 0
    while n:
        c += n%2
        n /= 2
    return c
hypothetical_model=np.empty([8939,256])
corr_matrix=np.empty([256,1998])
for j in range(0,8939):
    for k in range(0,256):
            #x=int(ciphertexts[j][slice(2,4)],16)
            #x=ciphertexts[j][slice(0,2)]
			#10th Byte after Shifting of Row becomes 8th Byte
            x=ciphertexts[j][slice(16,18)]
			x8=int(x,16)^k
            #print(x8,"\n")
            cipher9[j][k]=InvSbox[x8] #Here we are not considereing InvSR because we are getting 0th byte
            #x1=ciphertexts[j][slice(0,2)]
            x2=int(x,16)
            xor1=x2^int(cipher9[j][k])
            hamming_dist=number_of_ones(xor1)
            hypothetical_model[j][k]= hamming_dist
#Number of rows in Trace is 8938
#Number of rows in Hypothetical Model is 8939
#This was throwing error when running below code. So to correct that, dropped the last row.
hypothetical_model = hypothetical_model[:-1]
corr_matrix=np.empty([256,1998])
tr=pd.DataFrame(traces)
hy=pd.DataFrame(hypothetical_model)
for i in range (0,256):
    for j in range (0,1998):
            c1=hy.iloc[:,i]
            #print(c1)
            c2=tr.iloc[:,j]
            corr, pval=pearsonr(c1,c2)
            corr_matrix[i][j]=(abs)(corr)
from numpy import unravel_index
keyy,y=unravel_index(corr_matrix.argmax(), corr_matrix.shape)
print(keyy)
#print corr_matrix
import matplotlib.pyplot as plt
for i in range(256):
    if (i==keyy):
        plt.plot(range(1998),corr_matrix[i,:], 'r',linewidth=5)
    else:
        plt.plot(range(1998),corr_matrix[i,:], 'k', linewidth=2)
plt.show()

TabError: inconsistent use of tabs and spaces in indentation (3764344811.py, line 49)